In [2]:
//#r "nuget:PdfPig"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Core.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokens.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokenization.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Fonts.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.DocumentLayoutAnalysis.dll"
    
using UglyToad.PdfPig;
using UglyToad.PdfPig.Content;
using UglyToad.PdfPig.Core;
using UglyToad.PdfPig.DocumentLayoutAnalysis;
using UglyToad.PdfPig.DocumentLayoutAnalysis.WordExtractor;
using XPlot.Plotly;
using System.IO;
using System.Linq;

Invalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Core.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokens.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokenization.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Fonts.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.DocumentLayoutAnalysis.dll

## Sources
- [_The Document Spectrum for Page Layout Analysis_](https://ieeexplore.ieee.org/document/244677), Lawrence O’Gorman
- https://inside.mines.edu/~whoff/courses/EENG510/projects/2015/Hoch.pdf


- https://github.com/UglyToad/PdfPig
- https://devblogs.microsoft.com/cesardelatorre/using-ml-net-in-jupyter-notebooks/

## Steps
0. Open pdf document, extract words and preprocess
1. Estimate within-line and between-line spacing
2. Get lines
3. Get Paragraphs

In [5]:
string pdfName = "Random 2 Columns Lists Hyph.pdf";
//string pdfName = "Random 2 Columns Lists Hyph - Justified.pdf";
int pageNo = 1;

List<Word> wordsRaw = new List<Word>();
double width = 0;
double height = 0;

using (PdfDocument document = PdfDocument.Open(pdfName))
{
    var page = document.GetPage(pageNo);
    width = page.Width;
    height = page.Height;
    wordsRaw = page.GetWords(NearestNeighbourWordExtractor.Instance).ToList();
}

var words = new List<Word>();
// only keep non space words
foreach (var word in wordsRaw)
{
    if (string.IsNullOrWhiteSpace(word.Text.Trim())) continue;
    words.Add(word);
}